## Extract Poses from Amass Dataset

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from tqdm import tqdm



from human_body_prior.tools.omni_tools import copy2cpu as c2c

os.environ['PYOPENGL_PLATFORM'] = 'egl'

/home/user/miniconda3/envs/PriorMDM/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Please remember to download the following subdataset from AMASS website: https://amass.is.tue.mpg.de/download.php. Note only download the <u>SMPL+H G</u> data.
* ACCD (ACCD)
* HDM05 (MPI_HDM05)
* TCDHands (TCD_handMocap)
* SFU (SFU)
* BMLmovi (BMLmovi)
* CMU (CMU)
* Mosh (MPI_mosh)
* EKUT (EKUT)
* KIT  (KIT)
* Eyes_Janpan_Dataset (Eyes_Janpan_Dataset)
* BMLhandball (BMLhandball)
* Transitions (Transitions_mocap)
* PosePrior (MPI_Limits)
* HumanEva (HumanEva)
* SSM (SSM_synced)
* DFaust (DFaust_67)
* TotalCapture (TotalCapture)
* BMLrub (BioMotionLab_NTroje)

### Unzip all datasets. In the bracket we give the name of the unzipped file folder. Please correct yours to the given names if they are not the same.

### Place all files under the directory **./amass_data/**. The directory structure shoud look like the following:  
./amass_data/  
./amass_data/ACCAD/  
./amass_data/BioMotionLab_NTroje/  
./amass_data/BMLhandball/  
./amass_data/BMLmovi/   
./amass_data/CMU/  
./amass_data/DFaust_67/  
./amass_data/EKUT/  
./amass_data/Eyes_Japan_Dataset/  
./amass_data/HumanEva/  
./amass_data/KIT/  
./amass_data/MPI_HDM05/  
./amass_data/MPI_Limits/  
./amass_data/MPI_mosh/  
./amass_data/SFU/  
./amass_data/SSM_synced/  
./amass_data/TCD_handMocap/  
./amass_data/TotalCapture/  
./amass_data/Transitions_mocap/  

**Please make sure the file path are correct, otherwise it can not succeed.**

In [2]:
# Choose the device to run the body model on.
comp_device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
from my_smpl import MySMPL

body_model = MySMPL("body_models/smpl", gender="neutral", ext="pkl").to(comp_device)


In [6]:
import pickle
input_dir='dataset/aist/rotations/'
output_dir='dataset/aist/new_joints/'
output_dir2='dataset/aist/motion_database/'
os.makedirs(output_dir,exist_ok=True)
os.makedirs(output_dir2,exist_ok=True)

In [5]:
import binascii
import json
ex_fps = 60
with open("dataset/aist/scaling.txt")as f:
    id2scale={}
    for line in f.readlines():
        k,v=line.strip().split()
        v=float(v)
        id2scale[k]=v
print(id2scale["gBR_sBM_cAll_d04_mBR0_ch02"])
def amass_to_pose(src_path, save_path, save_path2, scale):
    rotations=np.load(src_path)
    head_pos=np.load(src_path.replace("part","root"))/scale
    assert rotations.shape[0]==head_pos.shape[0]
    with torch.no_grad():
        global_orient = torch.tensor(rotations[:,:3], dtype=torch.float32).to(comp_device) # controls the global root orientation
        body_pose = torch.tensor(rotations[:,3:], dtype=torch.float32).to(comp_device)
        head_pos = torch.tensor(head_pos, dtype=torch.float32).to(comp_device)  
        body = body_model(body_pose=body_pose, global_orient=global_orient)
        joints = body.joints+(head_pos-body.joints[:,15]).unsqueeze(dim=1)
        joints = joints[:,:22].cpu().numpy()
    floor_height = joints.min(axis=0).min(axis=0)[1]
    joints[:, :, 1] -= floor_height
    root_pose_init_xz = joints[0,0] * np.float32([1, 0, 1])
    joints = joints - root_pose_init_xz
    np.save(save_path, joints)
    out={"root_positions": binascii.b2a_base64(
        joints[:,0].flatten().astype(np.float32).tobytes()).decode("utf-8"),
           "rotations": binascii.b2a_base64(rotations.flatten().astype(np.float32).tobytes()).decode(
               "utf-8"),
           "dtype": "float32",
           "fps": 60,
           "mode": "axis_angle",
           "n_frames": rotations.shape[0],
           "n_joints": 24}
    with open(save_path2, "w") as f:
        json.dump(out,f, indent=4)

#amass_to_pose(input_dir+"gBR_sBM_cAll_d04_mBR0_ch01/part_0.npy", output_dir+"aist_gBR_0.npy", output_dir2+"aist_gBR_0.json", id2scale["gBR_sBM_cAll_d04_mBR0_ch01"])

94.1002


This will take a few hours for all datasets, here we take one dataset as an example

To accelerate the process, you could run multiple scripts like this at one time.

In [51]:
joints=np.load(output_dir+"aist_gBR_1.npy")
print(joints[0])
print(joints.min(0))

[[ 0.00000000e+00  9.62539554e-01  0.00000000e+00]
 [ 6.89899474e-02  8.74000907e-01  2.52156928e-02]
 [-6.59901351e-02  8.70687366e-01  7.70531595e-04]
 [ 3.38736176e-03  1.06396747e+00 -4.78853360e-02]
 [ 1.39923140e-01  5.06629348e-01  6.96647838e-02]
 [-6.97639361e-02  5.60965776e-01  2.28721835e-01]
 [ 2.11457908e-03  1.19719505e+00 -7.14192241e-02]
 [ 7.98219293e-02  1.92877173e-01 -1.72024779e-01]
 [-1.90642327e-02  1.64534926e-01  2.61054493e-01]
 [-1.34350359e-02  1.24759448e+00 -4.55850437e-02]
 [ 4.35162187e-02  8.80824327e-02 -9.62815583e-02]
 [-7.13228434e-02  1.33273482e-01  3.81337710e-01]
 [-5.07029295e-02  1.45860553e+00 -4.71442938e-03]
 [ 3.99623662e-02  1.37590218e+00  8.14872235e-03]
 [-9.85109732e-02  1.36722612e+00 -7.29220062e-02]
 [-8.15213695e-02  1.49395955e+00  6.37298748e-02]
 [ 9.23681110e-02  1.39777648e+00  8.59879926e-02]
 [-1.71095729e-01  1.39686704e+00 -1.37840554e-01]
 [ 1.02162808e-02  1.19080973e+00  2.22850628e-01]
 [-2.05817640e-01  1.22481632e+

In [7]:
mid=0
for path in tqdm(sorted(os.listdir(input_dir))):
    for p in sorted(os.listdir(os.path.join(input_dir,path))):
        if p.startswith("root_"):continue
        amass_to_pose(os.path.join(input_dir,path,p), output_dir+f"aist_{path.split('_')[0]}_{mid}.npy", \
                      output_dir2+f"aist_{path.split('_')[0]}_{mid}.json", id2scale[path])
        mid+=1

100%|██████████████████████████████████████| 1408/1408 [00:03<00:00, 367.80it/s]


The above code will extract poses from **AMASS** dataset, and put them under directory **"./pose_data"**

The source data from **HumanAct12** is already included in **"./pose_data"** in this repository. You need to **unzip** it right in this folder.

## Segment, Mirror and Relocate Motions